Resistance, Support and Candles Fractals

In [132]:
import pandas as pd
import pandas_ta as ta

In [133]:
import pandas as pd
import pandas_ta as ta
#df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, length=12)
df.tail()

df=df[0:2000]

In [134]:
df.reset_index(drop=True, inplace=True)
df.isna().sum()

Gmt time     0
open         0
high         0
low          0
close        0
volume       0
RSI         12
dtype: int64

In [135]:
df['RSI'] = ta.rsi(df.close, length=12)
df.tail()

,Gmt time,open,high,low,close,volume,RSI
1995,28.08.2003 06:00:00.000,1.08538,1.08575,1.08377,1.08498,3.199040e+07,40.227736
1996,28.08.2003 07:00:00.000,1.08489,1.08588,1.08459,1.08525,3.171900e+07,42.132895
1997,28.08.2003 08:00:00.000,1.08517,1.08755,1.08515,1.08668,3.163140e+07,51.132311
1998,28.08.2003 09:00:00.000,1.08659,1.08730,1.08486,1.08532,3.299760e+07,44.028268
1999,28.08.2003 10:00:00.000,1.08552,1.08584,1.08142,1.08169,3.128480e+07,31.347005


In [137]:
## Support and Resistance Functions

wick_threshold = 0.0001

def support(df1, l, n1, n2):
    if (df1.low[l-n1:l].min() < df1.low[l] or
        df1.low[l+1:l+n2+1].min() < df1.low[l]):
        return 0
    candle_body = abs(df1.open[l]-df1.close[l])
    lower_wick = min(df1.open[l], df1.close[l])-df1.low[l]
    if (lower_wick > candle_body) and (lower_wick > wick_threshold):
        return 1
    
    return 0

def resistance(df1, l, n1, n2):
    if (df1.high[l-n1:l].max() > df1.high[l] or
        df1.high[l+1:l+n2+1].max() > df1.high[l]):
        return 0
    
    candle_body = abs(df1.open[l]-df1.close[l])
    upper_wick = df1.high[l]-max(df1.open[l], df1.close[l])
    if (upper_wick > candle_body) and (upper_wick > wick_threshold):
        return 1
    
    return 0

In [138]:
def closeResistance(l,levels,lim, df):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return min(levels, key=lambda x:abs(x-df.high[l]))
    else:
        return 0
    
def closeSupport(l,levels,lim, df):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return min(levels, key=lambda x:abs(x-df.low[l]))
    else:
        return 0

In [139]:
def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'high'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'low'].min() > level  

In [140]:
def check_candle_signal(l, n1, n2, backCandles, df):
    ss = []
    rr = []
    for subrow in range(l-backCandles, l-n2):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    ss.sort() #keep lowest support when popping a level
    for i in range(1,len(ss)):
        if(i>=len(ss)):
            break
        if abs(ss[i]-ss[i-1])<=0.0001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])<=0.0001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])<=0.0001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, 150e-5, df)
    cS = closeSupport(l, rrss, 150e-5, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    # could we consider the average RSI for the trend momentum?
    if (cR and is_below_resistance(l,6,cR, df) and df.RSI[l-1:l].min()<45 ):#and df.RSI[l]>65
        return 1
    elif(cS and is_above_support(l,6,cS,df) and df.RSI[l-1:l].max()>55 ):#and df.RSI[l]<35
        return 2
    else:
        return 0

In [141]:
from tqdm import tqdm

n1 = 8
n2 = 6
backCandles = 140

signal = [0 for i in range(len(df))]

for row in tqdm(range(backCandles+n1, len(df)-n2)):
    signal[row] = check_candle_signal(row, n1, n2, backCandles, df)

df["signal"] = signal

100%|██████████| 1846/1846 [00:38<00:00, 48.23it/s]


In [142]:
df[df['signal']==1].count()

Gmt time    42
open        42
high        42
low         42
close       42
volume      42
RSI         42
signal      42
dtype: int64

In [143]:
df[ (df['signal']==1) | (df['signal']==2)]

,Gmt time,open,high,low,close,volume,RSI,signal
179,14.05.2003 08:00:00.000,1.14720,1.15260,1.14667,1.15203,2.759340e+07,52.947000,1
228,16.05.2003 09:00:00.000,1.14057,1.14217,1.13990,1.14198,2.669290e+07,53.093655,1
256,19.05.2003 13:00:00.000,1.16598,1.16761,1.16274,1.16736,2.604230e+07,61.195997,2
261,19.05.2003 18:00:00.000,1.16786,1.16801,1.16374,1.16585,1.466139e+08,52.400581,2
274,20.05.2003 07:00:00.000,1.16695,1.16749,1.16351,1.16433,2.704080e+07,46.666791,2
...,...,...,...,...,...,...,...,...
1923,25.08.2003 06:00:00.000,1.08710,1.08789,1.08669,1.08699,3.187730e+07,42.511745,1
1924,25.08.2003 07:00:00.000,1.08711,1.08831,1.08688,1.08805,3.129550e+07,49.068385,1
1955,26.08.2003 14:00:00.000,1.08165,1.08801,1.08092,1.08798,3.150230e+07,61.222415,1
1970,27.08.2003 05:00:00.000,1.08372,1.08684,1.08370,1.08665,3.104610e+07,52.983432,1


In [145]:
import numpy as np
def pointpos(x):
    if x['signal']==1:
        return x['high']+1e-4
    elif x['signal']==2:
        return x['low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [146]:
dfpl = df[100:300]
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800, 
    paper_bgcolor='black',
    plot_bgcolor='black')
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=8, color="MediumPurple"),
                name="Signal")
fig.show()

In [147]:
def check_candle_signal_plot(l, n1, n2, backCandles, df, proximity):
    ss = []
    rr = []
    for subrow in range(l-backCandles, l-n2):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    ss.sort() #keep lowest support when popping a level
    i = 0
    while i < len(ss)-1:
        if abs(ss[i]-ss[i+1]) <= proximity:
            # ss[i] = (ss[i]+ss[i+1])/2
            # del ss[i+1]
            del ss[i+1]
        else:
            i+=1

    rr.sort(reverse=True) # keep highest resistance when popping one
    i = 0
    while i < len(rr)-1:
        if abs(rr[i]-rr[i+1]) <= proximity:
            #rr[i] = (rr[i]+rr[i+1])/2
            #del rr[i+1]
            del rr[i]
        else:
            i+=1

    dfpl=df[l-backCandles-n1:l+n2+50]
    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

    c=0
    while (1):
        if(c>len(ss)-1 ):
            break
        fig.add_shape(type='line', x0=l-backCandles-n1, y0=ss[c],
                    x1=l,
                    y1=ss[c],
                    line=dict(color="MediumPurple",width=2), name="Support"
                    )
        c+=1

    c=0
    while (1):
        if(c>len(rr)-1 ):
            break
        fig.add_shape(type='line', x0=l-backCandles-n1, y0=rr[c],
                    x1=l,
                    y1=rr[c],
                    line=dict(color="Red",width=2), name="Resistance"
                    )
        c+=1    

    # fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
    #             marker=dict(size=5, color="MediumPurple"),
    #             name="Signal")

    fig.update_layout(
    autosize=False,
    width=1000,
    height=800,)
    
    fig.show()
 
 
    #----------------------------------------------------------------------
    cR = closeResistance(l, rr, 150e-5, dfpl)
    cS = closeSupport(l, ss, 150e-5, dfpl)
    #print(cR, is_below_resistance(l,6,cR, dfpl))
    if (cR and is_below_resistance(l,6,cR, dfpl) ):#and df.RSI[l]>65
        return 1
    elif(cS and is_above_support(l,6,cS,dfpl) ):#and df.RSI[l]<35
        return 2
    else:
        return 0

In [148]:
check_candle_signal_plot(231, 4,4,100,df,0.002)

1

VWAP and EMA

In [3]:
import pandas as pd
import pandas_ta as ta

df=pd.read_csv("BTCUSD_Candlestick_15_M_ASK_05.08.2019-29.04.2022.csv")

df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]

df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df["EMA"]=ta.ema(df.Close, length=100)

In [4]:
emasignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

In [53]:
VWAPsignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.VWAP[i]:
            dnt=0
        if df.Low[i]<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

In [54]:
def TotalEMASignal(l):
    myclosedistance = 100
    if (df.EMASignal[l]==2  
        and min(abs(df.EMA[l]-df.High[l]),abs(df.EMA[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.EMASignal[l]==1 
        and min(abs(df.EMA[l]-df.High[l]),abs(df.EMA[l]-df.Low[l]))<=myclosedistance):
            return 1

def TotalVWAPSignal(l):
    myclosedistance = 100
    if (df.VWAPSignal[l]==2 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.VWAPSignal[l]==1 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 1        

# Change signal Here VWAP and EMA
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalVWAPSignal(row)
df['TotalSignal'] = TotSignal

In [55]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-3
    elif x['TotalSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [56]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[150:400]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, line=dict(color='blue', width=1), name="VWAP")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [57]:
dfpl = df[:].copy()
# dfpl.reset_index(inplace=True)
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=5)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [58]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 0.8*self.data.ATR[-1]
        TPSLRatio = 2.5

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



KeyboardInterrupt: 

In [ ]:
bt.plot()

In [ ]:
class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.EMA[-1]
            tp1 = self.data.Close[-1] + (self.data.Close[-1]-self.data.EMA[-1])*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.EMA[-1]
            tp1 = self.data.Close[-1] - (self.data.EMA[-1]-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

In [ ]:
bt.plot()

In [ ]:
class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.VWAP[-1]
            tp1 = self.data.Close[-1] + (self.data.Close[-1]-self.data.VWAP[-1])*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.VWAP[-1]
            tp1 = self.data.Close[-1] - (self.data.VWAP[-1]-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

In [ ]:
bt.plot()

Bullish DOJI and Bullish Engulfing Candles

In [105]:
import yfinance as yf
import pandas as pd

dataF = pd.DataFrame(mt.copy_rates_range('EURUSD', 
                                            mt.TIMEFRAME_M5, 
                                            datetime(2022, 7, 1), 
                                            datetime.now()))

dataF['time']=pd.to_datetime(dataF['time'],unit='s')
dataF=dataF[['time', 'open', 'high', 'low', 'close', 'real_volume']]
dataF.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
dataF.set_index("Datetime", inplace=True)
dataF=dataF[dataF.High!=dataF.Low]

In [107]:
import pandas_ta as ta
my_bbands = ta.bbands(dataF.Close, length=30, std=1.5)
dataF=dataF.join(my_bbands)

TypeError: 'NoneType' object is not iterable

In [63]:
def bollinger_doji_signal(df): 
    #bullish signal
    if ( df.Close.iloc[-1] < df['BBL_30_1.5'].iloc[-1] and
       #df.Close.iloc[-1] > df['BBL_30_1.5'].iloc[-1] and df.Close.iloc[-1] < df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] < df.Open.iloc[-3] ):
        return 2
    
    #bearish signal
    elif ( df.Close.iloc[-1] > df['BBU_30_1.5'].iloc[-1] and
        #df.Close.iloc[-1] < df['BBU_30_1.5'].iloc[-1] and df.Close.iloc[-1] > df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] > df.Open.iloc[-3] ):
         return 1
    
    #nosignal
    else:
        return 0

signal = [0]*len(dataF)
for i in range(20,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= bollinger_doji_signal(df)
dataF["bollinger_doji_signal"] = signal

In [64]:
import numpy as np
def pointpos(x):
    if x['bollinger_doji_signal']==1:
        return x['High']+0.5e-3
    elif x['bollinger_doji_signal']==2:
        return x['Low']-0.5e-3
    else:
        return np.nan
dataF['pointpos'] = dataF.apply(lambda row: pointpos(row), axis=1)

In [65]:
dataF.reset_index(inplace=True)
#dataF[dataF["bollinger_doji_signal"]!=0]

In [66]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

st = 700
dfpl = dataF[st:st+250].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_30_1.5'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_30_1.5'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.update_layout(autosize=False, width=1000, height=600)
fig.show()

In [67]:
def SIGNAL():
    return dataF.bollinger_doji_signal

In [68]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class BreakOut(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = min(self.data.Low[-2], self.data.Low[-1])
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = max(self.data.High[-2], self.data.High[-1])
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dataF, BreakOut, cash=1000, margin=1/10)
stat = bt.run()
stat

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_19568\3037684933.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   68796.0
Duration                              68796.0
Exposure Time [%]                    0.318328
Equity Final [$]                   978.103075
Equity Peak [$]                    1016.63494
Return [%]                          -2.189692
Buy & Hold Return [%]                2.203005
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.581551
Avg. Drawdown [%]                   -1.942043
Max. Drawdown Duration                63849.0
Avg. Drawdown Duration           21292.666667
# Trades                                 58.0
Win Rate [%]                        32.758621
Best Trade [%]                       0.311856
Worst Trade [%]                     -0.328275
Avg. Trade [%]                    

In [69]:
bt.plot()

GridPlot(id='p1532', ...)

In [70]:
def bollinger_engulfing_signal(df): 
    #bullish signal
    if ( df.Close.iloc[-1] < df['BBL_30_1.5'].iloc[-1] and
        #df.Close.iloc[-1] > df['BBL_30_1.5'].iloc[-1] and df.Close.iloc[-1] < df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df.Open.iloc[-1] and
       df.Close.iloc[-2] < df.Open.iloc[-2] and
       df.Open.iloc[-1] < df.Close.iloc[-2] and
       df.Close.iloc[-1] > df.Open.iloc[-2] ):
        return 2
    
    #bearish signal
    elif ( df.Close.iloc[-1] > df['BBU_30_1.5'].iloc[-1] and
        #df.Close.iloc[-1] < df['BBU_30_1.5'].iloc[-1] and df.Close.iloc[-1] > df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df.Open.iloc[-1] and
       df.Close.iloc[-2] > df.Open.iloc[-2] and
       df.Open.iloc[-1] > df.Close.iloc[-2] and
       df.Close.iloc[-1] < df.Open.iloc[-2] ):
         return 1
    
    #nosignal
    else:
        return 0

signal = [0]*len(dataF)
for i in range(20,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= bollinger_engulfing_signal(df)
dataF["bollinger_engulfing_signal"] = signal

In [71]:
def pointpos(x):
    if x['bollinger_engulfing_signal']==1:
        return x['High']+1e-3
    elif x['bollinger_engulfing_signal']==2:
        return x['Low']-1e-3
    else:
        return np.nan
dataF['pointpos'] = dataF.apply(lambda row: pointpos(row), axis=1)

In [72]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

st = 200
dfpl = dataF[st:st+250].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_30_1.5'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_30_1.5'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.update_layout(autosize=False, width=1000, height=600)
fig.show()

In [73]:
def SIGNAL():
    return dataF.bollinger_engulfing_signal

In [74]:
class BreakOut(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Low[-2]
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.High[-2]
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dataF, BreakOut, cash=1000, margin=1/10)
stat = bt.run()
stat

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_19568\1416694760.py:22: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   68796.0
Duration                              68796.0
Exposure Time [%]                     1.35471
Equity Final [$]                   1159.81556
Equity Peak [$]                    1190.27226
Return [%]                          15.981556
Buy & Hold Return [%]                2.203005
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.521796
Avg. Drawdown [%]                   -0.533607
Max. Drawdown Duration                14906.0
Avg. Drawdown Duration            1109.269231
# Trades                                190.0
Win Rate [%]                        47.368421
Best Trade [%]                       0.276207
Worst Trade [%]                     -0.231471
Avg. Trade [%]                    

In [75]:
bt.plot()

GridPlot(id='p2570', ...)

Scalping Strategy with VWAP, Bolling Bands and RSI

In [109]:
import pandas as pd
import pandas_ta as ta

import pandas as pd
df=pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")

df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
len(df)

df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI']=ta.rsi(df.Close, length=16)
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

In [77]:
VWAPsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

In [78]:
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_14_2.0'][l]
        and df.RSI[l]<45):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_14_2.0'][l]
        and df.RSI[l]>55):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

In [83]:
df[df.TotalSignal!=0].count()

Open           0
High           0
Low            0
Close          0
Volume         0
VWAP           0
RSI            0
BBL_14_2.0     0
BBM_14_2.0     0
BBU_14_2.0     0
BBB_14_2.0     0
BBP_14_2.0     0
VWAPSignal     0
TotalSignal    0
dtype: int64

In [85]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [86]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=10400
dfpl = df[st:st+350]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

In [87]:
dfpl = df[:75000].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [89]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/10, commission=0.00)
stat = bt.run()
stat

Start                     2022-06-30 22:00:00
End                       2023-06-02 23:55:00
Duration                    337 days 01:55:00
Exposure Time [%]                         0.0
Equity Final [$]                        100.0
Equity Peak [$]                         100.0
Return [%]                                0.0
Buy & Hold Return [%]                2.203005
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [90]:
bt.plot(show_legend=False)

C:\Users\Utilisateur\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '1H'. See `Backtest.plot(resample=...)`

C:\Users\Utilisateur\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

C:\Users\Utilisateur\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'



GridPlot(id='p3392', ...)

Stop Loss VS Trail Stop VS ATR related SL && Trail SL

In [108]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")

df.columns = ['Local time', 'open', 'high', 'low', 'close', 'volume']

#Check if NA values are in data
df=df[df['volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)

import pandas_ta as pa
df['ATR'] = pa.atr(high=df.high, low=df.low, close=df.close, length=14)
df.tail(20)

,Local time,open,high,low,close,volume,ATR
123821,14.04.2023 01:00:00.000,1.10544,1.10636,1.10529,1.10636,9.459190e+06,0.001131
123822,14.04.2023 02:00:00.000,1.10635,1.10751,1.10631,1.10702,1.028026e+07,0.001135
123823,14.04.2023 03:00:00.000,1.10701,1.10734,1.10662,1.10703,8.111400e+06,0.001106
123824,14.04.2023 04:00:00.000,1.10702,1.10727,1.10675,1.10689,7.359250e+06,0.001064
123825,14.04.2023 05:00:00.000,1.10690,1.10746,1.10619,1.10742,9.466770e+06,0.001079
123826,14.04.2023 06:00:00.000,1.10742,1.10758,1.10630,1.10668,1.391234e+07,0.001093
123827,14.04.2023 07:00:00.000,1.10668,1.10714,1.10593,1.10653,1.533735e+07,0.001101
123828,14.04.2023 08:00:00.000,1.10653,1.10660,1.10539,1.10590,1.579984e+07,0.001109
123829,14.04.2023 09:00:00.000,1.10589,1.10644,1.10553,1.10626,1.048760e+07,0.001095
123830,14.04.2023 10:00:00.000,1.10623,1.10666,1.10562,1.10586,1.139424e+07,0.001091


In [97]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [98]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

# EurUSD set
mybodydiff = 0.000001
mybodydiffmin = 0.002

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<mybodydiff:
        bodydiff[row]=mybodydiff      

    bodydiffmin = mybodydiffmin
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0

def isStar(l):
    bodydiffmin = mybodydiffmin
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<mybodydiff:
        bodydiff[row]=mybodydiff
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    #!!!
    #lim=df.ATR[l]/2
    
    #diff between high and closest level among levels
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    #diff between higher body and closest level to high
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    #min body less than closest level to high
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    #low price less than closest level to high
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    #!!!
    #lim=df.ATR[l]/2
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [99]:
for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    #!!!! parameters
    myclosedistance = 150e-5 #EURUSD
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, myclosedistance) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, myclosedistance)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0


IndexError: list assignment index out of range

In [100]:
df['signal']=signal

ValueError: Length of values (68797) does not match length of index (68830)

In [ ]:
print(df[df['signal']==1].count(),df[df['signal']==2].count())

In [131]:
df.columns = ['Local time', 'Open', 'High', 'low', 'Close', 'Volume', 'ATR', 'signal']
#df=df.iloc[100:200]
#help(pa.atr)
df

ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [ ]:
def SIGNAL():
    return df.signal

In [ ]:
# Trader fixed SL and TP
from backtesting import Strategy, Backtest
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 550e-4 #EURUSD
            tp1 = self.data.Close[-1] + 600e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 550e-4 #EURUSD
            tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

In [ ]:
# ATR related SL and TP
from backtesting import Strategy, Backtest
import numpy as np


class MyCandlesStrat(Strategy): 
    atr_f = 0.2
    ratio_f = 1
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] + self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            tp1 = self.data.Close[-1] - self.data.ATR[-1]*self.ratio_f/self.atr_f
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

In [ ]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    sltr=500e-4 #EURUSD
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat

In [ ]:
#ATR distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    atr_f = 1
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.data.ATR[-1]/self.atr_f)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.data.ATR[-1]/self.atr_f)

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            self.sell(sl=sl1)
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.000)
stat = bt.run()
stat